In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Common Includes

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket'
!lscpu | grep 'Thread(s) per core'
!nvidia-smi

Dataset

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64

ds_val = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/val",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (331, 331)
)

def normalize_image(image, label):
    image = tf.keras.applications.nasnet.preprocess_input(image)
    #image = tf.cast(x=image, dtype=tf.float32)/127.5 - 1.0
    return image, label

ds_val = ds_val.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.prefetch(buffer_size=AUTOTUNE)

Model

In [ ]:
model_full = tf.keras.applications.nasnet.NASNetLarge(
    input_shape=None,
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
)

model_full.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),  #The model contains the softmax layer at the end
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

model_full.summary()

Evaluate

In [14]:
model_full.evaluate(ds_val, verbose=1)

Calculate Flops

In [16]:
!pip install keras-flops
from keras_flops import get_flops
flops = get_flops(model_full, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

Save Model

In [17]:
Model_Name = "NASNet_large_ImageNet_81_96_47_8GF"
os.environ["MODELNAME"] = Model_Name
os.environ["MODELNAMEZIP"] = Model_Name + ".zip"
model_full.save("./" + Model_Name)
!zip -r ./$MODELNAMEZIP ./$MODELNAME
!rm -rf ./$MODELNAME